Make my hand dirty with Quora challenge.This is my first competition.Please suggest me to do better and it if helpful then don't forget to up vote and leave you feedback  

In [126]:
#Import Initial Packages
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from subprocess import check_output
import gensim
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from textblob import TextBlob
from nltk.stem import WordNetLemmatizer
import re 
from collections import namedtuple
import multiprocessing
import datetime
import os

tokenizer = RegexpTokenizer(r'\w+')
stopwords = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

**Data Analysis and Natural Language processing**

In [127]:
df_train = pd.read_csv("../data/train.csv")
df_test = pd.read_csv("../data/test.csv")

In [232]:
df_train.head(10)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0
9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0


In [129]:
df_test.head(3)

,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?


In [143]:
df_train_set1 = df_train[["question1"]]
df_train_set2 = df_train[["question2"]]

df_test_set1 = df_test[["question1"]]
df_test_set2 = df_test[["question2"]]

In [145]:
df_train_set1.columns = ["question"]
df_train_set2.columns =["question"]
df_train_set = pd.concat([df_train_set1,df_train_set2],axis=0)
print("df_train_set_1 :",df_train_set1.shape)
print("df_train_set_2 :",df_train_set1.shape)
print("df_train_set :",df_train_set.shape)

df_train_set_1 : (404290, 1)
df_train_set_2 : (404290, 1)
df_train_set : (808580, 1)


In [234]:
df_train_set[7:8]

,question
7,How can I be a good geologist?


In [146]:
df_test_set1.columns = ["question"]
df_test_set2.columns =["question"]
df_test_set = pd.concat([df_test_set1,df_test_set2],axis=0)
print("df_test_set1 :",df_test_set1.shape)
print("df_test_set2 :",df_test_set2.shape)
print("df_test_set :",df_test_set.shape)

df_test_set1 : (2345796, 1)
df_test_set2 : (2345796, 1)
df_test_set : (4691592, 1)


In [179]:
dataset = pd.concat([df_train_set,df_test_set],axis=0)

In [183]:
dataset[404290:404291]

,question
0,What is the step by step guide to invest in sh...


In [185]:
df_train_set2.head(2)

,question
0,What is the step by step guide to invest in sh...
1,What would happen if the Indian government sto...


In [186]:
dataset.shape

(5500172, 1)

In [153]:
#Lemmatizing words.Ex - consider cats and cat are different word even they both are mostly similar cotext or reffer to similar things
print(lemmatizer.lemmatize("cats"))

cat


In [154]:
#Language Processing
def get_processed_text(text=""):
    """
    Remove stopword,lemmatizing the words and remove special character to get important content
    """
    clean_text = re.sub('[^a-zA-Z0-9 \n\.]', ' ', text)
    tokens = tokenizer.tokenize(clean_text)
    tokens = [lemmatizer.lemmatize(token.lower().strip()) for token in tokens
              if token not in stopwords and len(token) >= 2]
    return tokens

In [155]:
text = "What is the best phone to buy below under 15k?"
print ("Original Text : ",text)
processed_text = " ".join(get_processed_text(text))
print ("Processed Text : ",processed_text) #Remove special character(?),english stop words(is,the,by,to,in) 

Original Text :  What is the best phone to buy below under 15k?
Processed Text :  what best phone buy 15k


In [198]:
#Process and clean up traing set
alldocuments = []
analyzedDocument = namedtuple('AnalyzedDocument', 'words tags')       
keywords = []
i=0
for index,record in dataset.iterrows():
    question = str(record["question"])
    tokens = get_processed_text(question)
    words = tokens
    words_text = " ".join(words)
    words = gensim.utils.simple_preprocess(words_text)
    tags = [i]
    i=i+1
    alldocuments.append(analyzedDocument(words, tags))

In [202]:
alldocuments[5500171]

AnalyzedDocument(words=['how', 'get', 'rid', 'erectile', 'dysfunction'], tags=[5500171])

In [188]:
len(alldocuments)#5500172

5500172

In [208]:
def train_and_save_doc2vec_model(alldocuments,document_model="model1",m_iter=100,m_min_count=2,m_size=100,m_window=3):
            print ("Start Time : %s" %(str(datetime.datetime.now())))
            #Train Model
            cores = multiprocessing.cpu_count()
            abs_path = os.getcwd()
            saved_model_name = "20170531_A_doc_2_vec_%s" %(document_model)
            doc_vec_file = "%s" %(saved_model_name)
            if document_model == "model1":
                # PV-DBOW 
                model_1 = gensim.models.Doc2Vec(alldocuments,dm=0,workers=cores,size=m_size, window=m_window,min_count=m_min_count,iter=m_iter,dbow_words=1)
                model_1.save("%s" %(doc_vec_file))
                print ("model training completed : %s" %(doc_vec_file))
            elif document_model == "model2":
                # PV-DBOW 
                model_2 = gensim.models.Doc2Vec(alldocuments,dm=0,workers=cores,size=m_size, window=m_window,min_count=m_min_count,iter=m_iter,dbow_words=0)
                model_2.save("%s" %(doc_vec_file))
                print ("model training completed : %s" %(doc_vec_file))
            elif document_model == "model3":
                # PV-DM w/average
                model_3 = gensim.models.Doc2Vec(alldocuments,dm=1, dm_mean=1,size=m_size, window=m_window,min_count=m_min_count,iter=m_iter)
                model_3.save("%s" %(doc_vec_file))
                print ("model training completed : %s" %(doc_vec_file))

            elif document_model == "model4":
                # PV-DM w/concatenation - window=5 (both sides) approximates paper's 10-word total window size
                model_4 = gensim.models.Doc2Vec(alldocuments,dm=1, dm_concat=1,workers=cores, size=m_size, window=m_window,min_count=m_min_count,iter=m_iter)
                model_4.save("%s" %(doc_vec_file))
                print ("model training completed : %s" %(doc_vec_file))
            print ("Record count %s" %len(alldocuments))
            print ("End Time %s" %(str(datetime.datetime.now())))
            return model_1 

In [209]:
#Train model
model=train_and_save_doc2vec_model(alldocuments)

2017-05-31 09:16:34,092 : INFO : collecting all words and their counts
2017-05-31 09:16:34,095 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2017-05-31 09:16:34,135 : INFO : PROGRESS: at example #10000, processed 63650 words (1599817/s), 10075 word types, 10000 tags
2017-05-31 09:16:34,175 : INFO : PROGRESS: at example #20000, processed 127533 words (1632868/s), 14295 word types, 20000 tags
2017-05-31 09:16:34,210 : INFO : PROGRESS: at example #30000, processed 191209 words (1897790/s), 17420 word types, 30000 tags
2017-05-31 09:16:34,251 : INFO : PROGRESS: at example #40000, processed 254344 words (1598247/s), 19921 word types, 40000 tags
2017-05-31 09:16:34,290 : INFO : PROGRESS: at example #50000, processed 318241 words (1653576/s), 22274 word types, 50000 tags


Start Time : 2017-05-31 09:16:34.092810


2017-05-31 09:16:34,323 : INFO : PROGRESS: at example #60000, processed 381816 words (1952450/s), 24249 word types, 60000 tags
2017-05-31 09:16:34,362 : INFO : PROGRESS: at example #70000, processed 445296 words (1701029/s), 26051 word types, 70000 tags
2017-05-31 09:16:34,397 : INFO : PROGRESS: at example #80000, processed 509015 words (1887102/s), 27598 word types, 80000 tags
2017-05-31 09:16:34,434 : INFO : PROGRESS: at example #90000, processed 572592 words (1722269/s), 29069 word types, 90000 tags
2017-05-31 09:16:34,470 : INFO : PROGRESS: at example #100000, processed 636183 words (1815936/s), 30501 word types, 100000 tags
2017-05-31 09:16:34,511 : INFO : PROGRESS: at example #110000, processed 699206 words (1578500/s), 31825 word types, 110000 tags
2017-05-31 09:16:34,546 : INFO : PROGRESS: at example #120000, processed 762639 words (1829450/s), 33079 word types, 120000 tags
2017-05-31 09:16:34,590 : INFO : PROGRESS: at example #130000, processed 826343 words (1476776/s), 34308 

model training completed : 20170531_A_doc_2_vec_model1
Record count 5500172
End Time 2017-05-31 17:20:02.313545


In [226]:
model

In [210]:
document1=['what', 'step', 'guide', 'invest', 'share', 'market']
document2=['when', 'expect', 'cognizant', 'confirmation', 'mail']

In [220]:
test_data_1 = 'How can I be a good geologist?'  
processed_text = " ".join(get_processed_text(test_data_1))
print(processed_text)

how good geologist


In [222]:
inferred_vector = model.infer_vector(processed_text)  
print(inferred_vector)  
sims = model.docvecs.most_similar([inferred_vector], topn=1)  
print(sims)  

[  1.96833559e-03   1.46121439e-03  -2.03739083e-03  -3.50498222e-03
   4.89428500e-03   3.20025044e-03  -3.62292398e-03  -1.59739633e-03
  -3.17800953e-03  -2.04928895e-03  -1.86812738e-03  -3.27793648e-03
  -4.46182955e-03   2.97749625e-03  -3.01577005e-04  -3.60245211e-03
  -5.41726709e-04   3.69466236e-03  -2.63058022e-03  -3.39078461e-03
   2.53156433e-03  -1.39358116e-03  -2.41153641e-03   1.84121169e-03
  -1.64729892e-04   9.37261793e-04  -2.05767021e-04   4.41140775e-03
  -3.12785758e-03  -4.11768910e-03   2.46983394e-03  -2.92320526e-03
   4.54142597e-03   2.13000155e-03  -1.88999763e-03   5.94311627e-04
   1.16033445e-03   1.82679773e-03  -3.82362283e-03   2.41257786e-03
  -2.56166793e-03  -2.99570966e-03   4.08918690e-03   4.62542754e-03
   3.50586511e-03   4.65720706e-03   1.74934545e-03  -2.30912981e-03
   3.96768982e-03   1.00838311e-03  -3.76398885e-03  -1.11376995e-03
  -3.95250390e-04   4.68214089e-03  -3.05453199e-03   3.07961996e-03
   4.05656127e-03   1.50578519e-04

In [225]:
alldocuments[2358812]

AnalyzedDocument(words=['how', 'good', 'forced', 'air', 'hydronic', 'baseboard', 'heating'], tags=[2358812])

In [125]:
 model.docvecs[99]

array([ 0.01102276,  0.0927269 ,  0.01254428, -0.05821465,  0.13292679,
       -0.0933377 , -0.0142699 , -0.15039271,  0.00444514,  0.05072165,
        0.03503428,  0.00797918,  0.03998345, -0.09167188,  0.02238131,
        0.02811193, -0.08409538,  0.00095293,  0.13202424,  0.1106971 ,
       -0.05425756, -0.0872968 , -0.02404629, -0.11878752, -0.02755945,
        0.01420203, -0.07636143,  0.00732703, -0.1143554 , -0.11797629,
        0.13454536, -0.12882271, -0.1577732 , -0.02043041,  0.01598631,
       -0.02229547, -0.15821387, -0.05063676, -0.06798314,  0.05210823,
       -0.1048374 ,  0.13625818, -0.0421809 ,  0.14949328,  0.01139477,
        0.0312469 , -0.03258416, -0.07215652,  0.10761771,  0.09912831,
       -0.04151942, -0.11991806, -0.00139744,  0.01892133,  0.0453564 ,
        0.02569379,  0.01613218, -0.01641301, -0.08025321, -0.01272497,
        0.07867078, -0.08451847,  0.00274036,  0.07076716, -0.09373389,
        0.05038563, -0.06855009,  0.02141299,  0.04377241, -0.07

In [219]:
df_train.head(10)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0
9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0


In [27]:
ls -ltr

 驱动器 H 中的卷是 umbrA
 卷的序列号是 000A-DFCB

 H:\Quora\kernels 的目录



找不到文件


In [33]:
def get_question_similarity_score(question1="",question2=""):
    print ("question1 - ",question1)
    print ("question2 - ",question2)
    model_name = "%s" %("doc_2_vec_model1")
    model_saved_file = "%s" %(model_name)
    model = gensim.models.doc2vec.Doc2Vec.load(model_saved_file)
    
    question_token1 = get_processed_text(question1)
    tokenize_text1 = ' '.join(question_token1)
    tokenize_text1 = gensim.utils.simple_preprocess(tokenize_text1)
    infer_vector_of_question1 = model.infer_vector(tokenize_text1)
    
    print("tokenize_text1",tokenize_text1,"infer_vector_of_question1",infer_vector_of_question1)
    
    question_token2 = get_processed_text(question2)
    tokenize_text2 = ' '.join(question_token2)
    tokenize_text2 = gensim.utils.simple_preprocess(tokenize_text2)
    infer_vector_of_question2 = model.infer_vector(tokenize_text2)
    
    print("tokenize_text2",tokenize_text2,"infer_vector_of_question2",infer_vector_of_question2)
    #similarity_score = 1
    similarity_score = model.docvecs.most_similar(infer_vector_of_question1)
    msg= "question : %s model_name : %s " %(question,model_name)
   
    return similarity_score

In [32]:
test_id_list

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [31]:
similarity_score_list

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

# Doc2vec examle

输入文件Tweets_id_text.txt的格式就是每个doc 对应内容的分词，空格隔开，每个doc是一行

用TaggedLineDocument 实现，每个doc默认编号

博文doc2vec/paragraph2vec使用说明（二）中介绍 带多个标签的文档向量训练方法。

In [39]:
import gensim, logging
import os

logging.basicConfig(format = '%(asctime)s : %(levelname)s : %(message)s', level = logging.INFO)
sentences = gensim.models.doc2vec.TaggedLineDocument(alldocuments)
model = gensim.models.Doc2Vec(sentences, size = 100, window = 5)
model.save('review_pure_text_model.txt')
print (len(model.docvecs))
out = file('review_pure_text_vector.txt', 'w')
for idx, docvec in enumerate(model.docvecs):
    for value in docvec:
      out.write(str(value) + ' ')
    out.write('\n')
    print (idx)
    print (docvec)
out.close()

2017-05-30 14:25:45,802 : INFO : collecting all words and their counts


TypeError: don't know how to handle uri [AnalyzedDocument(words=['what', 'step', 'step', 'guide', 'invest', 'share', 'market', 'india'], tags=['1']), AnalyzedDocument(words=['what', 'story', 'kohinoor', 'koh', 'noor', 'diamond'], tags=['3']), AnalyzedDocument(words=['how', 'increase', 'speed', 'internet', 'connection', 'using', 'vpn'], tags=['5']), AnalyzedDocument(words=['why', 'mentally', 'lonely', 'how', 'solve'], tags=['7']), AnalyzedDocument(words=['which', 'one', 'dissolve', 'water', 'quikly', 'sugar', 'salt', 'methane', 'carbon', 'di', 'oxide'], tags=['9']), AnalyzedDocument(words=['astrology', 'capricorn', 'sun', 'cap', 'moon', 'cap', 'rising', 'say'], tags=['11']), AnalyzedDocument(words=['should', 'buy', 'tiago'], tags=['13']), AnalyzedDocument(words=['how', 'good', 'geologist'], tags=['15']), AnalyzedDocument(words=['when', 'use', 'instead'], tags=['17']), AnalyzedDocument(words=['motorola', 'company', 'can', 'hack', 'charter', 'motorolla', 'dcx'], tags=['19']), AnalyzedDocu

gensim/docs/notebooks/doc2vec-wikipedia.ipynb

https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/doc2vec-wikipedia.ipynb

In [ ]:
#Process and clean up traing set
alldocuments = []
analyzedDocument = namedtuple('AnalyzedDocument', 'words tags')       
keywords = []
for index,record in df_train_set[0:100].iterrows():
    qid = str(record["qid"])
    question = str(record["question"])
    tokens = get_processed_text(question)
    words = tokens
    words_text = " ".join(words)
    words = gensim.utils.simple_preprocess(words_text)
    tags = [qid]
    alldocuments.append(analyzedDocument(words, tags))

In [57]:
class TaggedWikiDocument(object):
    def __init__(self, wiki):
        self.wiki = wiki
        self.wiki.metadata = True
    def __iter__(self):
        for content, (page_id, title) in self.wiki.get_texts():
            yield TaggedDocument([c.decode("utf-8") for c in content], [title])

In [59]:
documents = TaggedWikiDocument(xxx)

In [62]:
pre = Doc2Vec(min_count=0)
pre.scan_vocab(documents)

2017-05-30 14:55:38,746 : INFO : collecting all words and their counts


AttributeError: 'Series' object has no attribute 'get_texts'

In [49]:
xxx=df_train_set.head(100)['question']
xxx

0     What is the step by step guide to invest in sh...
1     What is the story of Kohinoor (Koh-i-Noor) Dia...
2     How can I increase the speed of my internet co...
3     Why am I mentally very lonely? How can I solve...
4     Which one dissolve in water quikly sugar, salt...
5     Astrology: I am a Capricorn Sun Cap moon and c...
6                                   Should I buy tiago?
7                        How can I be a good geologist?
8                       When do you use シ instead of し?
9     Motorola (company): Can I hack my Charter Moto...
10    Method to find separation of slits using fresn...
11          How do I read and find my YouTube comments?
12                 What can make Physics easy to learn?
13          What was your first sexual experience like?
14    What are the laws to change your status from a...
15    What would a Trump presidency mean for current...
16                         What does manipulation mean?
17    Why do girls want to be friends with the g

In [43]:
def getColName(colNum, stri):
    LogInfo(str(colNum)+','+stri)
    colName = []
    for i in range(colNum):
        colName.append(stri + str(i))
    return colName

In [55]:
from gensim.models.doc2vec import Doc2Vec
#-- doc2vec model function
def doc2vec(documents, topicNum):
    texts = [[word for word in document.split(' ')] for document in documents]
    print(len(texts))
    #print(texts)
    model = Doc2Vec(texts, size=topicNum, window=8,min_count=18, seed = 1)#, sample = 1e-5, iter = 10,seed = 1)
    doc2vecFeature = np.zeros((len(texts), topicNum))
    for i in range(len(texts)):
        vec = model.docvecs[i] 
        doc2vecFeature[i, :] = round(t[1],5)
        return doc2vecFeature

In [56]:
vecFeature = doc2vec(xxx, 5)
vecFeature

2017-05-30 14:49:07,631 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2017-05-30 14:49:07,631 : INFO : collecting all words and their counts


100


AttributeError: 'list' object has no attribute 'words'

In [41]:
#-- get doc2vec feature
def getDoc2vecFeature(dim):
    train = pd.read_csv('../feature/trainQlist.csv', header = 0, sep = ",")
    test = pd.read_csv('../feature/testQlist.csv', header = 0, sep = ",")
    data = pd.concat([train, test], axis = 0)
    
    vecFeature = doc2vec(data['qlist'].values, dim)
    colName = getColName(dim, "qvec")
    vecFeature = pd.DataFrame(vecFeature, columns = colName)

    vecFeature['uid'] = data['uid'].values.T
    print(vecFeature.shape)
    #name = '../feature/d2v/d2vFeature'+str(dim)+'.csv'
    #vecFeature.to_csv(name, index = False)
    return vecFeature

**Final Submition yet to come.Shorty i will update**